In [ ]:
results_dict = {}

import pandas as pd
import matplotlib.pyplot as plt

input_df = pd.read_csv("./standings/Premier_1993-2022.csv")
input_df = input_df[input_df["Season_End_Year"] == 2022]
input_df["Match Date"] = pd.to_datetime(input_df["Date"], dayfirst=True, errors='coerce')
input_df = input_df[["HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Match Date"]]
teams = sorted(set(input_df["HomeTeam"]).union(input_df["AwayTeam"]))
for team in teams:
    results_dict[team] = []

def points(x):
    if x < 0:
        return 0, 3
    elif x > 0:
        return 3, 0
    else:
        return 1, 1

for _, row in input_df.iterrows():
    home, away = points(row['HomeGoals'] - row['AwayGoals'])
    results_dict[row['HomeTeam']].append((home, row['Match Date']))
    results_dict[row['AwayTeam']].append((away, row['Match Date']))

rows = []

for team_name, data in results_dict.items():
    df = pd.DataFrame(data, columns=['points', 'date'])
    df = df.sort_values("date")

    df["year"] = df["date"].dt.isocalendar().year
    df["week"] = df["date"].dt.isocalendar().week

    weekly = df.groupby(["year", "week"], as_index=False)["points"].sum()

    weekly["total_points"] = weekly["points"].cumsum()
    weekly["team"] = team_name

    rows.append(weekly[["team", "year", "week", "total_points"]])

long_df = pd.concat(rows, ignore_index=True)
table = long_df.pivot(index="team", columns=["year", "week"], values="total_points")
table = table.sort_index(axis=1)
table = table.ffill(axis=1).fillna(0)
table.columns = [f"{year}_{week}" for year, week in table.columns]

season_standings_df = table.reset_index()
season_standings_df.to_csv("./standings/weekly.csv", index=False)

df = season_standings_df.set_index("team")

round_cols_sorted = sorted(df.columns, key=lambda c: (int(c.split("_")[0]), int(c.split("_")[1])))

plt.figure(figsize=(14, 8))
for team in df.index:
    plt.plot(
        round_cols_sorted,
        df.loc[team, round_cols_sorted],
        linewidth=2,
        label=team
    )

week_numbers = list(range(len(round_cols_sorted)))  # index positions
tick_positions = [i for i in range(len(round_cols_sorted)) if i % 4 == 0]

plt.xticks(
    ticks=tick_positions,
    labels=[round_cols_sorted[i] for i in tick_positions],
    rotation=90
)

plt.title("EPL Points Over Time (2021-2022 Season)", fontsize=18)
plt.xlabel("ISO Week")
plt.ylabel("Points")
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()
